In [6]:
from ortools.sat.python import cp_model
import numpy as np

In [7]:
def create_data_model(path):
    #Store the data model of the problem
    data = {}
    with open (path, "r") as f:
        lines = f.readlines()
        line = list(map(int,lines[0].strip().split()))
        N = line[0]
        M = line[1]
        SIGMA = N + M
        K = line[2]
        p = np.zeros(2*SIGMA +1, dtype='int')
        q = np.zeros(2*SIGMA +1, dtype='int')
        Q = np.zeros(K+1, dtype='int')
        d = np.zeros((2*SIGMA +1, 2*SIGMA+1), dtype='int')

        line = list(map(int,lines[1].strip().split()))
        for m in range(1,2*SIGMA +1):
            if (m<=N):
                p[m] = 1
            elif (m> SIGMA and m <= SIGMA + N):
                p[m] = -1
            if (m > N and m <= SIGMA):
                q[m] = line[m-N-1]
            elif (m>SIGMA+N and m <=2*SIGMA):
                q[m] = line[m-SIGMA-N-1]*(-1)
            else :
                q[m] = 0
                

        line = list(map(int,lines[2].strip().split()))
        for k in range(1,K+1):
            Q[k] = line[k-1]

        for i in range (0, 2*SIGMA+1):
            line = list(map(int,lines[i+3].strip().split()))
            for j in range(0, 2*SIGMA+1):
                d[i][j] = line[j]

        data['N'] = N
        data['M'] = M
        data['q'] = q.tolist()
        data['p'] = p.tolist()
        data['Q'] = Q.tolist()
        data['d'] = d.tolist()
        data['K'] = K
        data['root'] = 0
        data['SIGMA'] = SIGMA
    return data

In [10]:
data = create_data_model("../../res/data.text")
N= data['N']
M = data['M']
t = data['p']
q = data['q']
Q = data['Q']
d = data['d']
K = data['K']
root = data['root']
SIGMA = data['SIGMA']


In [11]:
model = cp_model.CpModel()

R = {}
V = {}
D= {}
w = {}
T = {}


###########variables################
for i in range(2*SIGMA + K):
    R[i] = model.NewIntVar(0, 2*SIGMA + K - 1 , f'R[{i}]')# vector điểm tiếp theo được đi đến

for i in range(2*SIGMA + K): ########## Có vấn đề khi khởi tạo V D w T
    V[i] = model.NewIntVar(0,100, f'V[{i}]')
    D[i] = model.NewIntVar(0,100)
    w[i] = model.NewIntVar(0,100)
    T[i] = model.NewIntVar(0,100)

###########constraints################
model.AddAllDifferent(R)
for i in range(2*SIGMA + K):
    model.Add(R[i] >= 0)
    model.Add(R[i] <= 2*SIGMA + K -1)
model.Add(R[10] == 0)
for i in range(K):
    model.Add(D[i] == 0)
    model.Add(w[i] == i)
    model.Add(V[i] == 0)
    model.Add(T[i] == 0)

for i in range(1, 2*SIGMA + K):
    b = model.NewBoolVar('b')
    target_element_V =  model.NewIntVar(0, 100, 'tV')
    target_element_T =  model.NewIntVar(0, 1, 'tT')
    target_element_D =  model.NewIntVar(0, 100, 'tD')
    target_element_w =  model.NewIntVar(0, K-1, 'tw')
    target_element_d =  model.NewIntVar(0, 100, 'td')
    model.AddElement(R[i], V, target_element_V)
    model.AddElement(R[i], T, target_element_T)
    model.AddElement(R[i], D, target_element_D)
    model.AddElement(R[i], w, target_element_w)
    model.AddElement(R[i], d[i], target_element_d)

    model.Add(R[i] >= K).OnlyEnforceIf(b)
    model.Add(R[i] < K).OnlyEnforceIf(b.Not())
    model.Add(target_element_V == V[i] + q[i]).OnlyEnforceIf(b) ######
    model.Add(target_element_V == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_T == T[i] + t[i]).OnlyEnforceIf(b)
    model.Add(target_element_T == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_D == D[i] + target_element_d).OnlyEnforceIf(b)
    model.Add(target_element_D == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_w == w[i]).OnlyEnforceIf(b)
    model.Add(target_element_w == R[i]).OnlyEnforceIf(b.Not())

for i in range(K, SIGMA + K):
    model.Add(w[i] == w[i + SIGMA])
    model.Add(D[i] <= D[i + SIGMA])

for i in range(2*SIGMA + K):
    # target_element_Q =  model.NewIntVar(0, 1000, 'tQ')
    # model.AddElement(w[i], Q, target_element_Q)
    target_element_Q = Q[w[i]]
    model.Add(0 <= T[i])
    model.Add(T[i] <= 1)
    model.Add(V[i] <= target_element_Q)

max_D = model.NewIntVar(0, 100 ,'max_D')
for i in range(2*SIGMA + K):
    model.Add(max_D >= D[i])
model.Minimize(max_D)

solver = cp_model.CpSolver()

status = solver.Solve(model)
print("Objective value ",solver.Value(max_D))
# if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
#     print("Objective value ",solver.Value(max_D))
# else:
#     print('No solution found.')

IndexError: list index out of range